In [1]:
import os
os.environ["LANGGRAPH_DISABLE_DAG_RENDER"] = "1"
from langgraph.graph import StateGraph, START, END
from typing import TypedDict, Annotated
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from google import genai
from langchain_core.messages import BaseMessage, HumanMessage
from langgraph.graph.message import add_messages

In [2]:
class ChatState(TypedDict):
    messages: Annotated[list[BaseMessage], add_messages ]

In [3]:
load_dotenv()

True

In [4]:
import os
api_key = os.getenv("GEMINI_API_KEY2") # or os.getenv("GEMINI_API_KEY")
if not api_key:
	raise EnvironmentError("GEMINI_API_KEY2 (or GEMINI_API_KEY) not set in environment. Add it to your .env or export it.")

# client = genai.Client(api_key=api_key)

In [5]:
model = ChatOpenAI(
   model="gemini-2.5-flash",
   api_key=api_key,
   base_url="https://generativelanguage.googleapis.com/v1beta/openai"
)

In [6]:
def llm_chat(state: ChatState):

    msg= state['messages']

    response= model.invoke(msg)
    print(state)
    return {'messages': [response]}

In [29]:
graph= StateGraph(ChatState)

graph.add_node('chatbot', llm_chat)

graph.add_edge(START, "chatbot")
graph.add_edge("chatbot", END)

chatbotnoob= graph.compile()

In [24]:
chatbotnoob.invoke({'messages': 'hi my name is KK'})

{'messages': [HumanMessage(content='hi my name is KK', additional_kwargs={}, response_metadata={}, id='fd4fdae8-7510-4cbf-b0d9-a062a0ef9459')]}


{'messages': [HumanMessage(content='hi my name is KK', additional_kwargs={}, response_metadata={}, id='fd4fdae8-7510-4cbf-b0d9-a062a0ef9459'),
  AIMessage(content='Hi KK! Nice to meet you. How can I help you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 6, 'total_tokens': 229, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_provider': 'openai', 'model_name': 'gemini-2.5-flash', 'system_fingerprint': None, 'id': '3GYdaaHyIq27g8UPvd3C4QY', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--a1af78b2-6116-4a53-a056-be53f9593fdc-0', usage_metadata={'input_tokens': 6, 'output_tokens': 15, 'total_tokens': 229, 'input_token_details': {}, 'output_token_details': {}})]}

In [25]:
chatbotnoob.invoke({'messages': 'How are you'})

{'messages': [HumanMessage(content='How are you', additional_kwargs={}, response_metadata={}, id='2e1237cb-44c5-4e91-94d2-d49bf74633ca')]}


{'messages': [HumanMessage(content='How are you', additional_kwargs={}, response_metadata={}, id='2e1237cb-44c5-4e91-94d2-d49bf74633ca'),
  AIMessage(content='As an AI, I don\'t have feelings or a physical state, so I can\'t be "well" in the human sense.\n\nHowever, I am functioning perfectly and ready to assist you! How can I help you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 49, 'prompt_tokens': 4, 'total_tokens': 441, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_provider': 'openai', 'model_name': 'gemini-2.5-flash', 'system_fingerprint': None, 'id': '72YdadKmJoSGqfkP6u7HiAY', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--36668515-b3da-40cf-8cad-55c6bcb1b3cb-0', usage_metadata={'input_tokens': 4, 'output_tokens': 49, 'total_tokens': 441, 'input_token_details': {}, 'output_token_details': {}})]}

In [19]:
chatbotnoob.invoke({'messages': 'what is my name?'})

{'messages': [HumanMessage(content='what is my name?', additional_kwargs={}, response_metadata={}, id='d6f5a7ce-0d8c-4bc5-bcc2-ddd7d774b396'),
  AIMessage(content="As an AI, I don't have access to personal information about you, so I don't know your name.\n\nYou can tell me your name if you'd like me to refer to you by it!", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 6, 'total_tokens': 375, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_provider': 'openai', 'model_name': 'gemini-2.5-flash', 'system_fingerprint': None, 'id': 'emYdafLiGL6kjuMP6vrJmQc', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--044f0132-f350-4d26-b41b-008f3236965b-0', usage_metadata={'input_tokens': 6, 'output_tokens': 45, 'total_tokens': 375, 'input_token_details': {}, 'output_token_details': {}})]}

In [30]:
while True:

    user_msg= input("Type Here:  ")

    if user_msg.strip().lower() in ['exit', 'quit', 'bye']:
        print('chat ended')
        break
    response= chatbotnoob.invoke({'messages':[HumanMessage(content=user_msg)]})
    print("AI:", response['messages'][-1].content)


{'messages': [HumanMessage(content='hi', additional_kwargs={}, response_metadata={}, id='5ec59752-43e5-4685-9f5f-7db22ab78750')]}
AI: Hello! How can I help you today?
{'messages': [HumanMessage(content='what is the capital of Telangana', additional_kwargs={}, response_metadata={}, id='d204d080-46ce-494d-8ca2-1ead69538203')]}
AI: The capital of Telangana is **Hyderabad**.
{'messages': [HumanMessage(content='add 100  and 50', additional_kwargs={}, response_metadata={}, id='7002fa98-1e77-46cc-a148-c18f563067ec')]}
AI: 100 + 50 = 150
{'messages': [HumanMessage(content='now add 60', additional_kwargs={}, response_metadata={}, id='c5fd8065-3cbd-4a7f-8853-5c7136fb07c3')]}
AI: Okay, I can add 60!

What number would you like me to add it to?
{'messages': [HumanMessage(content='end', additional_kwargs={}, response_metadata={}, id='e03cfa73-b9f9-486e-9b93-7d05588d7591')]}
AI: Okay. Is that the end of our discussion for now, or is there anything else I can assist you with?
chat ended


#### no chat memory and it is failing to remember and state is starting fresh so we need to use  persistence. 
#### using persistence we can save the chat in memory(RAM) or in DB.

In [14]:
from langgraph.checkpoint.memory import MemorySaver

In [15]:
## adding check pointer as memory saver type
checkpointer= MemorySaver()
graph= StateGraph(ChatState)

graph.add_node('chatbot', llm_chat)

graph.add_edge(START, "chatbot")
graph.add_edge("chatbot", END)

chatbotnoob2= graph.compile(checkpointer = checkpointer)



In [ ]:
thread_id= '1'
while True:

    user_msg= input("Type Here:  ")

    if user_msg.strip().lower() in ['exit', 'quit', 'bye']:
        print('chat ended')
        break
    config= {'configurable': {'thread_id': thread_id}}
    response= chatbotnoob2.invoke({'messages':[HumanMessage(content=user_msg)]}, config=config)
    print("AI:", response['messages'][-1].content)


{'messages': [HumanMessage(content='', additional_kwargs={}, response_metadata={}, id='6effbf6c-d231-48c2-b762-db575cf23b93')]}
AI: The problem asks us to find the "special sum" of a given integer `N`. The special sum is defined recursively:
1. If `N` is a single-digit number (0-9), its special sum is `N`.
2. Otherwise, its special sum is the special sum of the sum of its digits.

This process continues until we arrive at a single-digit number. This concept is also known as the [digital root](https://en.wikipedia.org/wiki/Digital_root) of a number.

Let's break down how to implement this.

### Approach 1: Iterative Simulation

We can directly implement the definition by repeatedly summing the digits of a number until the sum becomes a single-digit number.

**Algorithm:**
1. If `N` is 0, return 0 (it's a single-digit number).
2. While `N` is greater than or equal to 10:
   a. Initialize a `digit_sum` to 0.
   b. Take a temporary variable `temp_N = N`.
   c. While `temp_N` is greater tha

In [ ]:
thread_id= '1'
while True:

    user_msg= input("Type Here:  ")

    if user_msg.strip().lower() in ['exit', 'quit', 'bye']:
        print('chat ended')
        break
    config= {'configurable': {'thread_id': thread_id}}
    response= chatbotnoob2.invoke({'messages':[HumanMessage(content=user_msg)]}, config=config)
    print("AI:", response['messages'][-1].content)


{'messages': [HumanMessage(content='Hi my name is KK', additional_kwargs={}, response_metadata={}, id='7da4bac6-0667-4e5a-8c31-dc6ef2aabe54'), AIMessage(content="Hi KK! It's nice to meet you. How can I help you today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 6, 'total_tokens': 474, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_provider': 'openai', 'model_name': 'gemini-2.5-flash', 'system_fingerprint': None, 'id': 'dWsdaZ70Au2q4-EPy_ak6AY', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--3349c44d-5c8f-468c-a78d-7726d912df4d-0', usage_metadata={'input_tokens': 6, 'output_tokens': 18, 'total_tokens': 474, 'input_token_details': {}, 'output_token_details': {}}), HumanMessage(content='what is my name?', additional_kwargs={}, response_metadata={}, id='8712d5f6-6ca3-4740-b63c-2e66a2e9de6a'), AIMessage(content='Your name is KK! You told me that at the beginning of our chat. 😊', addit

In [ ]:
chatbotnoob2.get_state(config=config)

StateSnapshot(values={'messages': [HumanMessage(content='Hi my name is KK', additional_kwargs={}, response_metadata={}, id='7da4bac6-0667-4e5a-8c31-dc6ef2aabe54'), AIMessage(content="Hi KK! It's nice to meet you. How can I help you today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 6, 'total_tokens': 474, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_provider': 'openai', 'model_name': 'gemini-2.5-flash', 'system_fingerprint': None, 'id': 'dWsdaZ70Au2q4-EPy_ak6AY', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--3349c44d-5c8f-468c-a78d-7726d912df4d-0', usage_metadata={'input_tokens': 6, 'output_tokens': 18, 'total_tokens': 474, 'input_token_details': {}, 'output_token_details': {}}), HumanMessage(content='what is my name?', additional_kwargs={}, response_metadata={}, id='8712d5f6-6ca3-4740-b63c-2e66a2e9de6a'), AIMessage(content='Your name is KK! You told me that at the beginning o

```
persistence in LnagGraph  refers to the ability to save and restore the state of a work flow over the time.

It helps to store all the values in workflow, even if we change the intermidiate steps it will be stored.

With Persistence help we can start our workflow from where it failed knows as fault tolerence 

with Checkpointers help, persistence feature is added in langgraph.

checkpoint: it is devided and stores the values. and after and before superstep  we can add checkpoints. 
checkpointers available: MemorySaver, Postgres, redis

Threads: creating the thread id for each interraction. so can retrive any interraction in chatbot. 

Persistance feature in langgraph gives us the options such as:
 - short term memory: we can save the chat in memory or in DB., 
 - fault tolerence that means we can resume our workflow from where it got crashed by running same thread id,
 - Human in the loop: Workflow can be interrupted (temporary suspend), lets say for some human
 approval for moving forward with further workflow items.
 - Time Travel: we can replay the work, can go back to perticular step. it helps in debugging. 
   TO go back at perticular node and generate response from that node we can use checkpoint_id present in each response. and with only passing thread id and checkpoint id  we can generate responses from perticular step.


Streaming: In LLM, streaming means the model starts sending tokens (words) as soon as they are generated, instead  of waiting for the entire response to be ready before returning it.

```